In [1]:
#Designate your file locations:

attributes_data_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Inputs/'
daily_data_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Inputs/'
intermediate_output_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Intermediate/'
aggregated_output_location=r'C:/Users/Jaketosis/OneDrive/TAMU/MS Stats/STAT626/Project/Data/Intermediate/'

#Designate your file names: 

attributes_name='Attributes_DataFrame.csv'
daily_name='Daily_DataFrame.csv'
intermediate_output_name='transformed_data.csv'
total_aggregated_output_name='aggregated_daily.csv'

In [2]:
from datetime import date
import pandas as pd
import numpy as np
import cpi

c:\Users\Jaketosis\Anaconda3\lib\site-packages\cpi\__init__.py:42: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [ ]:
#Only run this if import cpi gives you a warning
#cpi.update()

In [ ]:
attributes_data=pd.read_csv(attributes_data_location+attributes_name)
daily_data=pd.read_csv(daily_data_location+daily_name)

In [ ]:
# Get datetime equivalent of the Date column
daily_data['dt']=pd.to_datetime(daily_data['Date'])

# Get the first date each title is shown
first_dates=daily_data.groupby(['Movie_Title'])[['dt']].min()
attributes_data['first_date']=attributes_data['Title'].map(first_dates['dt'])



#I know I could use the dt column to do this but I wrote this first
daily_data['year']=daily_data['Date'].str[:4].astype(int)
daily_data['month']=daily_data['Date'].str[5:7].astype(int)

daily_data=daily_data.loc[(daily_data['year']>=2005)&(daily_data['year']<=2019)]

#daily_data

In [ ]:
split=attributes_data.Genres.str.split(';',expand=True)
split.columns=["Genre "+str(colname) for colname in list(split.columns)]
list(split.columns)
attributes_data=attributes_data.join(split)

In [ ]:
merged_data=daily_data.merge(attributes_data,how='left',left_on='Movie_Title',right_on='Title')
merged_data['daily_pre_adjust']=merged_data['Daily']
# Get the overall ranking of the producers based on this data set
producer_agg=merged_data.groupby(['Distributor'])['Daily'].sum().sort_values(ascending=False).reset_index()
producer_agg['rank']=producer_agg['Daily'].rank(ascending=False)
producer_agg.set_index('Distributor',inplace=True,drop=True)

merged_data['Producer_Rank']=merged_data['Distributor'].map(producer_agg['rank'])
merged_data

In [ ]:
#This takes like 4 minutes to run, you can skip this cell if you already have the intermediate output data

to_transform_years=merged_data['year'].to_list()
to_transform_months=merged_data['month'].to_list()
to_tranform_vals=merged_data['Daily'].to_list()

new_list=[]
loop_list=[(to_transform_years[j], to_transform_months[j],to_tranform_vals[j]) for j in range(0,len(to_tranform_vals))]
for i_year,i_month,i in loop_list:
    append=cpi.inflate(i,date(i_year,i_month,1),to=date(2022,12,31))
    new_list=new_list+[append]

merged_data['Daily']=pd.Series(new_list)

In [ ]:
#Filter the unnecessary columns out

In [ ]:
#We want an intermediate output of our data before aggregating for use in R or a visualization software
merged_data.to_csv(intermediate_output_location+intermediate_output_name)

In [20]:
#start here if you want to skip the 4.5 minute long inflation adjusting loop
merged_data=pd.read_csv(intermediate_output_location+intermediate_output_name)

C:\Users\Jaketosis\AppData\Local\Temp\ipykernel_1184\3457788233.py:2: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data=pd.read_csv(intermediate_output_location+intermediate_output_name)


In [21]:
#merged_data.head()
merged_data.tail()

,Unnamed: 0,Movie_Title,Date,Daily,Theaters,Rank,dt,year,month,Title,...,Genre 1,Genre 2,Genre 3,Genre 4,Genre 5,Genre 6,Genre 7,Genre 8,daily_pre_adjust,Producer_Rank
203519,203519,Ashfall (2019),2019-12-27,11683.666254,2.0,33,2019-12-27,2019,12,Ashfall (2019),...,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,10116,120.0
203520,203520,Ashfall (2019),2019-12-28,18449.474569,2.0,32,2019-12-28,2019,12,Ashfall (2019),...,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,15974,120.0
203521,203521,Ashfall (2019),2019-12-29,16511.436612,2.0,32,2019-12-29,2019,12,Ashfall (2019),...,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,14296,120.0
203522,203522,Ashfall (2019),2019-12-30,11106.181762,2.0,32,2019-12-30,2019,12,Ashfall (2019),...,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,9616,120.0
203523,203523,Ashfall (2019),2019-12-31,16966.494392,2.0,30,2019-12-31,2019,12,Ashfall (2019),...,Adventure,Thriller,NaN,NaN,NaN,NaN,NaN,NaN,14690,120.0


In [22]:
#Check the range of the dates in the dataset. Notice that June 2021 was way too early for Movie theaters to be recovering during COVID. 
print(min(pd.to_datetime(merged_data['Date'])),max(pd.to_datetime(merged_data['Date'])))

2005-01-01 00:00:00 2019-12-31 00:00:00


In [23]:
merged_data.columns

Index(['Unnamed: 0', 'Movie_Title', 'Date', 'Daily', 'Theaters', 'Rank', 'dt',
       'year', 'month', 'Title', 'Domestic', 'International', 'Budget',
       'Distributor', 'MPAA-Rating', 'Runtime', 'Genres', 'first_date',
       'Genre 0', 'Genre 1', 'Genre 2', 'Genre 3', 'Genre 4', 'Genre 5',
       'Genre 6', 'Genre 7', 'Genre 8', 'daily_pre_adjust', 'Producer_Rank'],
      dtype='object')

In [24]:
# provide a tuple where the first element is the list of columns at which we are aggregating and the second column is the measure that is being aggregated
agg_list=[
    (['dt'],'Daily','total_agg'),
    (['dt','Distributor','Producer_Rank'],'Daily','studio_agg'),
    (['dt','Genre 0'],'Daily','genre_agg'),

]

merged_data['week']=pd.to_datetime(merged_data['dt']).dt.isocalendar().week
merged_data['year']=pd.to_datetime(merged_data['dt']).dt.isocalendar().year

date_dim=['']

for agg_dims,agg_measure,outname in agg_list:

    aggregated_data=merged_data.groupby(agg_dims).agg({agg_measure:'max'})
    aggregated_weekly=merged_data.groupby(['year','week']+agg_dims[1:]).agg({'Daily':['sum','min','max','mean']}).reset_index()
    
    aggregated_weekly['formatted_date'] = aggregated_weekly['year'] * 1000 + aggregated_weekly['week'] * 10 + 0
    aggregated_weekly['date'] = pd.to_datetime(aggregated_weekly['formatted_date'], format='%Y%W%w')
    aggregated_data.rename(columns={agg_measure:'max'},inplace=True)
    aggregated_data['min']=merged_data.groupby(agg_dims).agg({agg_measure:'min'})
    aggregated_data['mean']=merged_data.groupby(agg_dims).agg({agg_measure:'mean'})
    aggregated_data['sum']=merged_data.groupby(agg_dims).agg({agg_measure:'sum'})

    
    aggregated_data.reset_index(inplace=True)
    aggregated_data['dt']=pd.to_datetime(aggregated_data['dt'])
    #aggregated_data['dt']=pd.to_datetime(aggregated_data['dt'].map(lambda x: str(x)+'-0'),format="%Y%W-%w")
    #Write out your code to be consumed in R or a visualization software

    aggregated_data['week']=aggregated_data['dt'].dt.isocalendar().week
    aggregated_data['year']=aggregated_data['dt'].dt.isocalendar().year

    
    aggregated_weekly.to_csv(aggregated_output_location+outname+'_weekly_'+total_aggregated_output_name)
    aggregated_data.to_csv(aggregated_output_location+outname+'_'+total_aggregated_output_name)

In [25]:
aggregated_data

,dt,Genre 0,max,min,mean,sum,week,year
0,2005-01-01,Action,4.243895e+06,1.213490e+04,1.846840e+06,1.477472e+07,53,2004
1,2005-01-01,Adventure,8.905832e+06,2.084583e+04,3.016398e+06,1.206559e+07,53,2004
2,2005-01-01,Biography,6.897768e+06,9.198684e+04,1.694965e+06,1.016979e+07,53,2004
3,2005-01-01,Comedy,2.843763e+07,3.783500e+03,3.830034e+06,4.213037e+07,53,2004
4,2005-01-01,Crime,6.232420e+06,6.232420e+06,6.232420e+06,6.232420e+06,53,2004
...,...,...,...,...,...,...,...,...
42694,2019-12-31,Biography,1.498952e+06,4.677624e+02,4.269466e+05,3.415572e+06,1,2020
42695,2019-12-31,Comedy,3.376906e+06,2.136693e+02,5.485535e+05,4.936982e+06,1,2020
42696,2019-12-31,Crime,2.340100e+06,2.562645e+04,6.768505e+05,2.707402e+06,1,2020
42697,2019-12-31,Drama,4.737168e+06,3.166925e+03,8.199354e+05,4.919612e+06,1,2020


In [26]:
aggregated_weekly.grouby(['fomrmatted_date'])[[col for col in aggregated_weekly.columns where col not in ['year','week']+]]

year week    Genre 0         Daily                              \
                                     sum           min           max   
0     2004   53     Action  2.482213e+07  8.533497e+03  4.243895e+06   
1     2004   53  Adventure  2.117149e+07  1.477293e+04  8.905832e+06   
2     2004   53  Biography  1.714543e+07  7.890878e+04  6.897768e+06   
3     2004   53     Comedy  6.857602e+07  2.930617e+03  2.843763e+07   
4     2004   53      Crime  1.025590e+07  4.023476e+06  6.232420e+06   
...    ...  ...        ...           ...           ...           ...   
6249  2020    1  Biography  5.976166e+06  3.638152e+02  1.498952e+06   
6250  2020    1     Comedy  9.433266e+06  2.136693e+02  3.376906e+06   
6251  2020    1      Crime  5.673850e+06  1.715937e+04  2.522036e+06   
6252  2020    1      Drama  1.008283e+07  2.945171e+03  4.938455e+06   
6253  2020    1     Horror  3.460749e+05  1.977307e+03  2.022004e+05   

                   formatted_date       date  
              mean                            
0     1.551383e+06        2004530 2005-01-09  
1     2.646436e+06        2004530 2005-01-09  
2     1.428786e+06        2004530 2005-01-09  
3     3.117092e+06        2004530 2005-01-09  
4     5.127948e+06        2004530 2005-01-09  
...            ...            ...        ...  
6249  3.735104e+05        2020010 2020-01-12  
6250  5.240703e+05        2020010 2020-01-12  
6251  7.092312e+05        2020010 2020-01-12  
6252  8.402359e+05        2020010 2020-01-12  
6253  8.651873e+04        2020010 2020-01-12  

[6254 rows x 9 columns]